# 🧩 `parse_markdown_examples_to_json(text: str)`

Парсит текст с примерами, содержащими блоки `**Input:**` и `**Output:**`, и возвращает их в виде словаря. Этот формат используется у LeetCode.

---

## 📥 Аргументы

| Параметр | Тип   | Описание                             |
|----------|-------|--------------------------------------|
| `text`   | `str` | Текст, содержащий примеры в формате: |
| -        |       | `**Example N:**`                     |
| -        |       | `**Input:** <входные данные>`        |
| -        |       | `**Output:** <выходные данные>`      |

---

## 📤 Возвращает

Тип: `dict`

Структура возвращаемого словаря:

```json
{
  "examples": [
    {
      "input": {...},     // Входные данные (dict)
      "output": ...       // Выходные данные (любой тип)
    }
  ]
}
```

---


# 🔻 Вставьте пример для конвертирования в JSON

In [7]:
example_block = """
**Example 1:**

**Input:** nums = [-1,0,3,5,9,12], target = 9
**Output:** 4
**Explanation:** 9 exists in nums and its index is 4

**Example 2:**

**Input:** nums = [-1,0,3,5,9,12], target = 2
**Output:** -1
**Explanation:** 2 does not exist in nums so return -1
"""

## 1. Функция-обработчик

In [8]:
import re
import ast
import json

def parse_markdown_examples_to_json(text: str):
    examples = []
    # Разделяем текст на блоки по шаблону **Example N:**
    raw_blocks = re.split(r"\*\*Example \d+:\*\*", text)
    for block in raw_blocks:
        # Ищем **Input:** ... **Output:** в каждом блоке
        input_match = re.search(r"\*\*Input:\*\*\s*(.*?)\s*(?:\n|$)", block)
        output_match = re.search(r"\*\*Output:\*\*\s*(.*?)\s*(?:\n|$)", block)

        if not input_match or not output_match:
            continue

        input_text = input_match.group(1).strip()
        output_text = output_match.group(1).strip().rstrip(",")

        # Парсинг входных данных (input)
        try:
            input_expr = "{" + re.sub(r"(\w+)\s*=", r'"\1":', input_text) + "}"
            input_dict = ast.literal_eval(input_expr)
        except Exception as e:
            print(f"⚠ Ошибка в input '{input_text}': {e}")
            continue

        # Подготовка output-текста
        output_text = output_text.replace("true", "True").replace("false", "False")

        # Парсинг выходных данных (output)
        try:
            output = ast.literal_eval(output_text)
        except Exception as e:
            print(f"⚠ Ошибка в output '{output_text}': {e}")
            continue

        # Добавляем пример в список
        examples.append({
            "input": input_dict,
            "output": output
        })

    return {"examples": examples}



## 2. Парсим примеры из текста

In [9]:
parsed = parse_markdown_examples_to_json(example_block)
print(parsed)

{'examples': [{'input': {'nums': [-1, 0, 3, 5, 9, 12], 'target': 9}, 'output': 4}, {'input': {'nums': [-1, 0, 3, 5, 9, 12], 'target': 2}, 'output': -1}]}


## 3. Преобразуем результат в JSON-строку с красивым форматированием

In [10]:
pretty = json.dumps(parsed, indent=2, ensure_ascii=False)
print(pretty)

{
  "examples": [
    {
      "input": {
        "nums": [
          -1,
          0,
          3,
          5,
          9,
          12
        ],
        "target": 9
      },
      "output": 4
    },
    {
      "input": {
        "nums": [
          -1,
          0,
          3,
          5,
          9,
          12
        ],
        "target": 2
      },
      "output": -1
    }
  ]
}


## 4. Убираем лишние пробелы и переносы строк внутри массивов

In [11]:
pretty = re.sub(r'\[\s+([^\[\]]+?)\s+\]', lambda m: '[' + m.group(1).replace('\n', '').replace(' ', '') + ']', pretty)
print(pretty)

{
  "examples": [
    {
      "input": {
        "nums": [-1,0,3,5,9,12],
        "target": 9
      },
      "output": 4
    },
    {
      "input": {
        "nums": [-1,0,3,5,9,12],
        "target": 2
      },
      "output": -1
    }
  ]
}


# 🔺 Заберите результат в буфер обмена